# Spark DataFrames to Pandas conversion

## toPandas with Arrow

In [1]:
from pyspark.sql import SparkSession
ss = SparkSession.builder.getOrCreate()

from pyspark.sql.functions import rand
from pyspark.sql import Row
df = ss.sparkContext.range(1 << 22).map(lambda e: Row(uid=e)).toDF().withColumn("x", rand())
df.printSchema()

25/03/17 01:16:14 WARN Utils: Your hostname, macbook-dllllb.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
25/03/17 01:16:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/17 01:16:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/17 01:16:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


root
 |-- uid: long (nullable = true)
 |-- x: double (nullable = false)



In [4]:
ss.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

%time pdf = df.toPandas()

CPU times: user 5.36 s, sys: 1.45 s, total: 6.81 s
Wall time: 9.85 s


In [5]:
ss.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

%time pdf = df.toPandas()

CPU times: user 34 ms, sys: 76.4 ms, total: 110 ms
Wall time: 1.91 s


## toPandas using mapPartitions

In [5]:
from pyspark.sql import SparkSession
ss = SparkSession.builder.getOrCreate()

from pyspark.sql.functions import rand
from pyspark.sql import Row
df = ss.sparkContext.range(1 << 22).map(lambda e: Row(uid=e)).toDF().withColumn("x", rand())
df = df.repartition(100)
df.printSchema()

root
 |-- uid: long (nullable = true)
 |-- x: double (nullable = false)



In [7]:
import pandas as pd

def map_to_pandas(rdds):
    return [pd.DataFrame(list(rdds))]

%time parts = df.rdd.mapPartitions(map_to_pandas).collect()
%time pdf = pd.concat(parts)

CPU times: user 73 ms, sys: 55.8 ms, total: 129 ms
Wall time: 10.5 s
CPU times: user 54.4 ms, sys: 49.5 ms, total: 104 ms
Wall time: 104 ms
